In [2]:
import numpy as np
import pandas as pd
import cv2
import sys
from datetime import datetime
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from IPython.display import clear_output 

In [3]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name} [{handedness[0].index}]",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [4]:
# Use OpenCV’s VideoCapture to start capturing from the webcam.
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

In [5]:
model_path = './models/hand_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

class storage:
    def __init__(self):
        self.image = np.zeros((480, 640), dtype=np.uint8)

        self.size = 100
        self.hand_data = [{'x':np.zeros(shape=self.size), 'y':np.zeros(shape=self.size), 'z':np.zeros(shape=self.size)},{'x':np.zeros(shape=self.size), 'y':np.zeros(shape=self.size), 'z':np.zeros(shape=self.size)},{},{},{}]
        self.index = 0

    def get_annotated_image(self):
        return self.image
    
    def save_annotated_image(self, img):
        self.image = img

    def print_data(self):
        print(self.hand_data)        

    def save_data(self, i, x, y, z):
        self.hand_data[i]['x'][self.index] = x
        self.hand_data[i]['y'][self.index] = y
        self.hand_data[i]['z'][self.index] = z

        if(self.index < self.size-1):
            self.index += 1
        else:
            self.index = 0


# 이미지 저장하는 클래스
image_storage = storage()
# 데이터 저장하는 클래스
data_storage = storage()


In [6]:

# Create a hand landmarker instance with the live stream mode:
def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    # print('hand landmarker result: {}'.format(result))
    
    hand_landmarks_list = result.hand_landmarks
    handedness_list = result.handedness
    for i in range(len(hand_landmarks_list)):
        #print(hand_landmarks_list[i][0].x)
        hll = hand_landmarks_list[i][0]
        data_storage.save_data(i, hll.x, hll.y, hll.z)
        #hand_data[i] = hand_landmarks_list[i]
        
    detection_result = result
    mp_image = output_image
    annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), detection_result)
    image_storage.save_annotated_image(annotated_image)

    
# Create a hand landmarker instance with the image mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result,
    num_hands=5,
    min_hand_detection_confidence = 0.5,
    min_hand_presence_confidence = 0.5,
    min_tracking_confidence = 0.5)


In [7]:
with HandLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.

    # Create a loop to read the latest frame from the camera using VideoCapture#read()
    while webcam.isOpened():
        status, frame = webcam.read()
        frame_timestamp_ms = int(webcam.get(cv2.CAP_PROP_POS_MSEC))

        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
        # Send live image data to perform hand landmarks detection.
        # The results are accessible via the `result_callback` provided in
        # the `HandLandmarkerOptions` object.
        # The hand landmarker must be created with the live stream mode.
        landmarker.detect_async(mp_image, frame_timestamp_ms)
        
        
        if status:
            cv2.imshow("test", image_storage.get_annotated_image())
            #cv2.imshow("test", annotated_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cv2.destroyAllWindows()

    data_storage.print_data()

[{'x': array([0.49617603, 0.60651314, 0.46361467, 0.60092777, 0.43805119,
       0.59953117, 0.41975254, 0.60237676, 0.41215414, 0.61130708,
       0.40425432, 0.61723775, 0.39664418, 0.61349082, 0.3909643 ,
       0.6095506 , 0.38543391, 0.6171124 , 0.38388163, 0.6178171 ,
       0.38128483, 0.62126946, 0.38005793, 0.62362862, 0.37908053,
       0.63289392, 0.37762684, 0.6338436 , 0.37453201, 0.63066369,
       0.37244728, 0.61808348, 0.3679114 , 0.61255342, 0.36608076,
       0.61013567, 0.35962117, 0.61123562, 0.35297751, 0.62030381,
       0.34629962, 0.62472916, 0.33351216, 0.6227653 , 0.32071754,
       0.61547512, 0.32473153, 0.61913073, 0.62480634, 0.62200594,
       0.62887216, 0.63621795, 0.63384295, 0.63267356, 0.62816626,
       0.62890708, 0.62871516, 0.62524426, 0.6184445 , 0.61941391,
       0.6195758 , 0.6237269 , 0.62063336, 0.60476792, 0.62296838,
       0.60997051, 0.62334538, 0.61463338, 0.6232841 , 0.61636817,
       0.6158061 , 0.61063224, 0.61335558, 0.60449767, 

In [8]:
# 웹캠의 연결을 끊어준다.
# webcam.release() 